In [1]:
%%time
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torchvision.datasets import CIFAR10
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.autograd import Variable
import torch.nn.functional as F
from pathlib import Path

Wall time: 2.07 s


In [2]:
working_dir = Path('.')
DATA_PATH = Path("./Data/Paderborn files")

In [3]:
pwd

'C:\\Users\\Firdaus\\Belajar\\Paderborn'

In [4]:
%%time
data = np.load(DATA_PATH/'raw_signal_data.npy')
labels = np.load(DATA_PATH/'raw_signal_labels.npy')

Wall time: 11.4 s


In [5]:
x=data[:,0:1600]

In [6]:
def mean(data,no_elements):
    X=np.zeros((data.shape[0],data.shape[1]))
    for i in range(data.shape[1]-no_elements+1):
        X[:,i]=np.mean(data[:,i:i+no_elements],axis=1)
    return X.astype(np.float16)
def median(data,no_elements):
    X=np.zeros((data.shape[0],data.shape[1]))
    for i in range(data.shape[1]-no_elements+1):
        X[:,i]=np.median(data[:,i:i+no_elements],axis=1)
    return X.astype(np.float16)
def sig_image(data,size):
    X=np.zeros((data.shape[0],size,size))
    for i in range(data.shape[0]):
        X[i]=(data[i,:].reshape(size,size))
    return X.astype(np.float16)

In [7]:
%%time
channel_mean=(mean(x,10)).astype(np.float16)
x_m=sig_image(channel_mean,40)
channel_median=(median(x,10)).astype(np.float16)
x_md=sig_image(x,40)

Wall time: 25.5 s


In [8]:
x_n=sig_image(x,40)

In [9]:
x_n.shape

(58000, 40, 40)

In [10]:
x_m.shape

(58000, 40, 40)

In [11]:
X=np.stack((x_n,x_m,x_md),axis=1).astype(np.float16)

In [12]:
X.shape

(58000, 3, 40, 40)

In [13]:
%%time
from sklearn.model_selection import train_test_split
trainx, testx, trainlabel, testlabel = train_test_split(X, labels, test_size=0.2, random_state=20)

Wall time: 3.09 s


In [14]:
sig_train, sig_test = trainx,testx
lab_train, lab_test = trainlabel,testlabel

In [15]:
sig_train = torch.from_numpy(sig_train)
sig_test = torch.from_numpy(sig_test)
lab_train= torch.from_numpy(lab_train)
lab_test = torch.from_numpy(lab_test)

In [16]:
sig_train.shape

torch.Size([46400, 3, 40, 40])

In [17]:
import torch.utils.data as data_utils
batch_size = 128 
train_tensor = data_utils.TensorDataset(sig_train, lab_train) 
train_loader = data_utils.DataLoader(dataset = train_tensor, batch_size = batch_size, shuffle = True)

In [18]:
batch_size = 1024
test_tensor = data_utils.TensorDataset(sig_test, lab_test) 
test_loader = data_utils.DataLoader(dataset = test_tensor, batch_size = batch_size, shuffle = False)

In [19]:
sig_train.size()

torch.Size([46400, 3, 40, 40])

In [20]:
sig_test.size()

torch.Size([11600, 3, 40, 40])

In [25]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=4,stride=1,padding = 1)
        self.mp1 = nn.MaxPool2d(kernel_size=4,stride=2)
        self.conv2 = nn.Conv2d(32,64, kernel_size=4,stride =1)
        self.mp2 = nn.MaxPool2d(kernel_size=4,stride=2)
        self.fc1= nn.Linear(2304,256)
        self.dp1 = nn.Dropout(p=0.2)
        self.fc2 = nn.Linear(256,10)

    def forward(self, x):
        in_size = x.size(0)
        x = F.relu(self.mp1(self.conv1(x)))
        x = F.relu(self.mp2(self.conv2(x)))
        x = x.view(in_size,-1)
        x = F.relu(self.fc1(x))
        x = self.dp1(x)
        x = self.fc2(x)
        
        return F.log_softmax(x, dim=1)

In [26]:
cnn = CNN().double()

In [27]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(cnn.parameters(), lr=0.001)

In [28]:
num_epochs = 5

In [29]:
total_step = len(train_loader)
loss_list = []
acc_list = []
for epoch in range(num_epochs):
    for i, (signals, labels) in enumerate(train_loader):
        optimizer.zero_grad()
        # Run the forward pass
        signals=signals
        labels=labels
        outputs = cnn(signals.double())
        loss = criterion(outputs, labels.long())
        
        loss_list.append(loss.item())

        # Backprop and perform Adam optimisation
        
        loss.backward()
        optimizer.step()
        # Track the accuracy
        total = labels.size(0)
        _, predicted = torch.max(outputs.data, 1)
        correct = (predicted == labels.long()).sum().item()
        acc_list.append(correct / total)

        if (epoch+1) % 5 == 0 or epoch==0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Train Accuracy: {:.2f}%'
                  .format(epoch + 1, num_epochs, i + 1, total_step, loss.item(),
                          (correct / total) * 100))
        

Epoch [1/5], Step [1/363], Loss: 2.3245, Train Accuracy: 0.78%
Epoch [1/5], Step [2/363], Loss: 1.8126, Train Accuracy: 42.19%
Epoch [1/5], Step [3/363], Loss: 1.3959, Train Accuracy: 35.16%
Epoch [1/5], Step [4/363], Loss: 1.1324, Train Accuracy: 41.41%
Epoch [1/5], Step [5/363], Loss: 1.3164, Train Accuracy: 37.50%
Epoch [1/5], Step [6/363], Loss: 1.1250, Train Accuracy: 37.50%
Epoch [1/5], Step [7/363], Loss: 1.3880, Train Accuracy: 33.59%
Epoch [1/5], Step [8/363], Loss: 1.3750, Train Accuracy: 29.69%
Epoch [1/5], Step [9/363], Loss: 1.1103, Train Accuracy: 40.62%
Epoch [1/5], Step [10/363], Loss: 1.2737, Train Accuracy: 42.19%
Epoch [1/5], Step [11/363], Loss: 1.2393, Train Accuracy: 39.06%
Epoch [1/5], Step [12/363], Loss: 1.1529, Train Accuracy: 46.09%
Epoch [1/5], Step [13/363], Loss: 1.1363, Train Accuracy: 41.41%
Epoch [1/5], Step [14/363], Loss: 1.1152, Train Accuracy: 32.03%
Epoch [1/5], Step [15/363], Loss: 1.0959, Train Accuracy: 39.84%
Epoch [1/5], Step [16/363], Loss: 1

Epoch [1/5], Step [127/363], Loss: 0.5379, Train Accuracy: 75.00%
Epoch [1/5], Step [128/363], Loss: 0.5953, Train Accuracy: 72.66%
Epoch [1/5], Step [129/363], Loss: 0.6084, Train Accuracy: 65.62%
Epoch [1/5], Step [130/363], Loss: 0.5287, Train Accuracy: 76.56%
Epoch [1/5], Step [131/363], Loss: 0.5767, Train Accuracy: 74.22%
Epoch [1/5], Step [132/363], Loss: 0.4591, Train Accuracy: 78.12%
Epoch [1/5], Step [133/363], Loss: 0.5753, Train Accuracy: 73.44%
Epoch [1/5], Step [134/363], Loss: 0.5433, Train Accuracy: 73.44%
Epoch [1/5], Step [135/363], Loss: 0.5147, Train Accuracy: 78.12%
Epoch [1/5], Step [136/363], Loss: 0.5949, Train Accuracy: 71.09%
Epoch [1/5], Step [137/363], Loss: 0.5594, Train Accuracy: 75.78%
Epoch [1/5], Step [138/363], Loss: 0.5092, Train Accuracy: 79.69%
Epoch [1/5], Step [139/363], Loss: 0.5499, Train Accuracy: 75.78%
Epoch [1/5], Step [140/363], Loss: 0.5743, Train Accuracy: 73.44%
Epoch [1/5], Step [141/363], Loss: 0.5602, Train Accuracy: 73.44%
Epoch [1/5

KeyboardInterrupt: 

In [ ]:
total_step = len(test_loader)
print(total_step)
loss_list_test = []
acc_list_test = []
with torch.no_grad():
    for i, (signals, labels) in enumerate(test_loader):
        # Run the forward pass
        signals=signals
        labels=labels
        outputs = cnn(signals.double())
        loss = criterion(outputs, labels.long())
        loss_list_test.append(loss.item())
        if epoch%10 ==0:
            print(loss)
        total = labels.size(0)
        _, predicted = torch.max(outputs.data, 1)
        correct = (predicted == labels.long()).sum().item()
        acc_list_test.append(correct / total)
        if (epoch) % 1 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
                  .format(epoch + 1, num_epochs, i + 1, total_step, loss.item(),
                          (correct / total) * 100))

In [26]:
# if you need to save
torch.save(cnn,'cnnTC3_fold3_45.pth')

/home/cmu/anaconda3/lib/python3.7/site-packages/torch/serialization.py:402: UserWarning: Couldn't retrieve source code for container of type CNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
